In [1]:
path = "C:\\Users\\dmehri\\Documents\\DATA\\"

import pandas as pd
import numpy as np

def blight_model():
    
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier


    from sklearn.model_selection import train_test_split
    #from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
    #create data
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification, make_blobs
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import roc_curve, auc
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import GridSearchCV

    
      
        
    test = pd.read_csv(path + "test.csv")
    train = pd.read_csv(path + "train.csv",encoding = 'ISO-8859-1',low_memory=False )
    address = pd.read_csv(path + "addresses.csv",low_memory=False )
    latlon = pd.read_csv(path + "latlons.csv",low_memory=False )
    
    
    #DROP NAs
    #print ("len before drop NA", len(train) )
    train = train.dropna(subset = ['compliance']).reset_index(drop=True)
    #print ( "len after drop NA", len(train) )
    #print ("len of test dataset", len(test))
    
    #print ("Count the class labels")
    #print ("Compliance")
    #print  ( (train["compliance"]==1).sum() )
    #print ("No compliance")
    #print ( (train["compliance"]==0).sum() )
    #print 
    
    def CleanAndTransformData(df):
        #REMOVE FEATURES
        #DROP DATES
        df = df.drop('hearing_date', 1)
        df = df.drop('ticket_issued_date', 1)
        #ADDRESS    
        df = df.drop('mailing_address_str_number', 1)
        df = df.drop('mailing_address_str_name', 1)
        df = df.drop('city', 1)
        df = df.drop('state', 1)
        df = df.drop('zip_code', 1)
        df = df.drop('non_us_str_code', 1)
        df = df.drop('country', 1)
        df = df.drop('violator_name', 1)
        df = df.drop('violation_street_number', 1)
        df = df.drop('violation_street_name', 1)
        #DROP ASSORTED
        df = df.drop('inspector_name', 1)
        df = df.drop('violation_zip_code', 1)
        df = df.drop('clean_up_cost', 1)
        df = df.drop('admin_fee', 1)
        df = df.drop('state_fee', 1)
        df = df.drop('grafitti_status', 1)
        #PERHAPS TOO MANY CATEGORIES
        df = df.drop('violation_code', 1)
        df = df.drop('violation_description', 1)
        #CATEGORIES DON'T MATCH WITH TEST DATA
        df = df.drop('agency_name', 1)
        df = df.drop('disposition', 1)

        #ADD FEATURES
        #INSERT LAT/LON
        lat = latlon.set_index('address')['lat'].to_dict()
        lon = latlon.set_index('address')['lon'].to_dict()
        address["lat"] = address["address"].map(lat)
        address["lon"] = address["address"].map(lon)
        lat = address.set_index('ticket_id')['lat'].to_dict()
        lon = address.set_index('ticket_id')['lon'].to_dict()
        #map lat/lon to dfing dataset
        df["lat"] = df["ticket_id"].map(lat)
        df["lon"] = df["ticket_id"].map(lon)   
        
        return df

  
    train = CleanAndTransformData(train)
    test = CleanAndTransformData(test)

    #FEATURES NOT IN TEST DATA
    train = train.drop('payment_amount', 1)
    train = train.drop('payment_date', 1)
    train = train.drop('payment_status', 1)
    train = train.drop('balance_due', 1)
    train = train.drop('collection_status', 1)
    train = train.drop('compliance_detail', 1)
    
    #CREATE MODEL
    #train = train.drop('ticket_id', 1)
    
    model_selection = []
    
    def AccuracyAndAUC(model, clf, X_test, y_test, a, b):
        a = "(" + a + ","
        b = b + ")"
        accuracy_param = a + b
        clf_predicted = clf.predict(X_test)
        print ("Accuracy parameters", a, b)
        print('Accuracy on training set: {:.2f}'.format(clf.score(X_train, y_train)))
        print('Accuracy on test set: {:.2f}'.format(accuracy_score(y_test, clf_predicted)))
        print('Precision: {:.2f}'.format(precision_score(y_test, clf_predicted)))
        print('Recall: {:.2f}'.format(recall_score(y_test, clf_predicted)))
        print('F1: {:.2f}'.format(f1_score(y_test, clf_predicted)))
    
        #print(classification_report(y_test, clf_predicted, target_names=['not 1', '1']))
        
        #y_score_gb = clf.fit(X_train, y_train).decision_function(X_test)
        #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_gb)
        #roc_auc_gb = auc(fpr_lr, tpr_lr)
        #print ("AUC:", round(roc_auc_gb, 2))
        
        pred_prob = clf.predict_proba(X_test)[:,1]
        roc=roc_auc_score(y_test, pred_prob)
        print ('AUC:', round(roc, 2))

        
        #clf_predicted = clf.predict(X_test)
        #confusion = confusion_matrix(y_test, clf_predicted)

        #print('Classifier Confusion Matrix\n', confusion)
        print()
        
        model_selection.append([model,accuracy_param, clf.score(X_train, y_train), accuracy_score(y_test, clf_predicted),
                               precision_score(y_test, clf_predicted),recall_score(y_test, clf_predicted), 
                                f1_score(y_test, clf_predicted), roc])
        
        
        
    def AccuracySVM(clf, X_test, y_test, a, b):
        a = "(" + a + ","
        b = b + ")"
        clf_predicted = clf.predict(X_test)
        print ()
        print ("Accuracy on these parameters", a, b)
        print('Accuracy on test set: {:.2f}'.format(accuracy_score(y_test, clf_predicted)))
        print('Precision: {:.2f}'.format(precision_score(y_test, clf_predicted)))
        print('Recall: {:.2f}'.format(recall_score(y_test, clf_predicted)))
        print('F1: {:.2f}'.format(f1_score(y_test, clf_predicted)))
        
        #pred_prob = clf.predict_proba(X_test)[:,1]
        #roc=roc_auc_score(y_test, pred_prob)
        #print ('AUC:', round(roc, 2))
        
        y_score_gb = clf.fit(X_train, y_train).decision_function(X_test)
        fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_gb)
        roc_auc_gb = auc(fpr_lr, tpr_lr)
        print ("AUC:", round(roc_auc_gb, 2))


        clf_predicted = clf.predict(X_test)
        confusion = confusion_matrix(y_test, clf_predicted)

        print('Classifier Confusion Matrix\n', confusion)
        
        
    def PlotROC(clf, X_test, y_test):
        y_score_gb = clf.fit(X_train, y_train).decision_function(X_test)
        fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_gb)
        roc_auc = auc(fpr_lr, tpr_lr)
        #PLOT
        plt.figure()
        plt.xlim([-0.01, 1.00])
        plt.ylim([-0.01, 1.01])
        plt.plot(fpr_lr, tpr_lr, lw=3, label='ROC curve (area = {:0.2f})'.format(roc_auc))
        plt.xlabel('False Positive Rate', fontsize=16)
        plt.ylabel('True Positive Rate', fontsize=16)
        plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
        plt.legend(loc='lower right', fontsize=13)
        plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
        plt.axes().set_aspect('equal')
        plt.show()
        
        return plt.show()
    
    
    #MODEL
    y = train["compliance"]
    X = train[["fine_amount", "late_fee", "discount_amount", "judgment_amount", "lat", "lon"] ]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    #fillna with mean
    X_train = X_train.fillna(X_train.mean())
    
    #test baseline using dummy classifier
    dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    
    print ("Baseline Accuracy: Dummy majority score:", round(dummy_majority.score(X_test, y_test), 2) )
    print ()
    
    #RANDOM FORESTS
    print ("Random Forest")
    clf = RandomForestClassifier(random_state = 0)
    clf.fit(X_train, y_train)
    model = "Random Forest"
    a = "default"
    b = "default"
    AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    print ("Random Forest changing params")
    max_depth_list = [10,12,14, 16, 18]
    
    #max_features = sqrt(number of features)
    for max_d in max_depth_list:
        clf = RandomForestClassifier(max_features = 2,max_depth = max_d, random_state = 0)
        clf.fit(X_train, y_train)
        a = "max features:" + str(2)
        b = "max depth:" + str(max_d)
        AccuracyAndAUC(model, clf, X_test, y_test, a, b)

    
    
    
    #GRADIENT BOOSTED DECISION TREES
    print ("GRADIENT BOOSTED DECISION TREES")
    print ("Gradient boosting default parameters")
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train, y_train)
    model = "Gradient Boosted Dec Trees"
    a = "default"
    b = "default"
    AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    #PlotROC(clf, X_test, y_test)
    
    print ()
    print ("Adjusting gradient boosted learning rate and depth")
    learn_rate = [0.01, 0.1, 1]
    max_depth = [2,4,6]
    
    for learn in learn_rate:
        for depth in max_depth:
            clf = GradientBoostingClassifier(learning_rate = learn, max_depth = depth, random_state = 0)
            clf.fit(X_train, y_train)
            a = "learning rate:" + str(learn)
            b = "maximum depth:" + str(depth)
            AccuracyAndAUC(model, clf, X_test, y_test, a, b)
            

         
    #LOGISTIC REGRESSION
    print ()
    print ("LOGISTIC REGRESSION")
    print ("Default parameters")
    clf = LogisticRegression().fit(X_train, y_train)
    clf.fit(X_train, y_train)
    model = "Logistic Regression"
    a = "default"
    b = "default"
    AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    print ()
    print ("Varying regularization c parameter")
    this_C_list = [0.1, 1, 100]
    b = "none"
    for this_C in this_C_list:
        a = "C reg param:" + str(this_C)
        clf = LogisticRegression(C=this_C).fit(X_train, y_train)
        AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    
    #K-NEAREST NEIGHBORS
    print ()
    print ("KNN")
    clf = KNeighborsClassifier(n_neighbors = 5)
    clf.fit(X_train, y_train)
    model = "KNN"
    a = "none"
    b = "none"
    AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    print()
    print ("KNN with MinMax Scaling")
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    # we must apply the scaling to the test set that we computed for the training set
    X_test_scaled = scaler.transform(X_test)
    clf = KNeighborsClassifier(n_neighbors = 5)
    clf.fit(X_train_scaled, y_train)
    a = "MinMax Scaled"
    b = "none"
    AccuracyAndAUC(model, clf, X_test_scaled, y_test, a, b)
    

    #NAIVE BAYES
    print ()
    print ("NAIVE BAYES")
    clf = GaussianNB().fit(X_train, y_train)
    model = "Naive Bayes"
    a = "default"
    b = "default"
    AccuracyAndAUC(model, clf, X_test, y_test, a, b)
    
    print()
    
    #SVM
    print ("SUPPORT VECTOR MACHINES")
    #print ("With default RBF Kernal and MinMax Scaling")
    model = "SVM"
    a = "MinMax Scalaing"
    b = "Grid Search"
    accuracy_param = "(" + a + "," + b + ")"
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    #clf = SVC(random_state = 0)
    #clf.fit(X_train_scaled, y_train)
    #AccuracySVM(clf, X_test_scaled, y_test, a, b)
    
    print ("SVM with RBF kernal and grid search optimizing for AUC")
    
    clf = SVC(kernel='rbf')
    grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
    
    grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    grid_clf_auc.fit(X_train_scaled, y_train)
    y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test_scaled) 

    print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    
   
        
    model_selection.append([model,accuracy_param, " ", grid_clf_auc.best_score_,
                               " ", " ", " ", roc_auc_score(y_test, y_decision_fn_scores_auc)])
    
    return model_selection

df = blight_model()


df2 = pd.DataFrame(df, columns=('Model','Parameters', 'Training Score', 'Test Score', 'Precision', 'Recall', 'F1', 'AUC'))
#df.to_csv(path + "train_edit.csv", index=False)

df2 = df2.round(3)

Baseline Accuracy: Dummy majority score: 0.93

Random Forest
Accuracy parameters (default, default)
Accuracy on training set: 0.98
Accuracy on test set: 0.93
Precision: 0.49
Recall: 0.25
F1: 0.33
AUC: 0.74

Random Forest changing params
Accuracy parameters (max features:2, max depth:10)
Accuracy on training set: 0.94
Accuracy on test set: 0.93
Precision: 0.89
Recall: 0.11
F1: 0.20
AUC: 0.77

Accuracy parameters (max features:2, max depth:12)
Accuracy on training set: 0.94
Accuracy on test set: 0.93
Precision: 0.85
Recall: 0.12
F1: 0.20
AUC: 0.78

Accuracy parameters (max features:2, max depth:14)
Accuracy on training set: 0.94
Accuracy on test set: 0.93
Precision: 0.79
Recall: 0.13
F1: 0.22
AUC: 0.78

Accuracy parameters (max features:2, max depth:16)
Accuracy on training set: 0.94
Accuracy on test set: 0.93
Precision: 0.75
Recall: 0.14
F1: 0.24
AUC: 0.78

Accuracy parameters (max features:2, max depth:18)
Accuracy on training set: 0.95
Accuracy on test set: 0.93
Precision: 0.68
Recall

In [3]:
df2

,Model,Parameters,Training Score,Test Score,Precision,Recall,F1,AUC
0,Random Forest,"(default,default)",0.981553,0.927,0.486815,0.248963,0.329444,0.739
1,Random Forest,"(max features:2,max depth:10)",0.936861,0.935,0.885559,0.112379,0.199448,0.775
2,Random Forest,"(max features:2,max depth:12)",0.938145,0.935,0.847716,0.115491,0.203287,0.778
3,Random Forest,"(max features:2,max depth:14)",0.941031,0.934,0.790598,0.127939,0.220238,0.778
4,Random Forest,"(max features:2,max depth:16)",0.944041,0.935,0.753704,0.140733,0.237179,0.777
5,Random Forest,"(max features:2,max depth:18)",0.947469,0.934,0.677761,0.15491,0.252181,0.776
6,Gradient Boosted Dec Trees,"(default,default)",0.935794,0.935,0.940063,0.103043,0.185728,0.775
7,Gradient Boosted Dec Trees,"(learning rate:0.01,maximum depth:2)",0.934259,0.934,0.945255,0.0895574,0.163613,0.753
8,Gradient Boosted Dec Trees,"(learning rate:0.01,maximum depth:4)",0.935643,0.935,0.95098,0.100622,0.181989,0.765
9,Gradient Boosted Dec Trees,"(learning rate:0.01,maximum depth:6)",0.935718,0.934,0.950495,0.0995851,0.180282,0.770


### BEST MODEL

In [4]:
def best_model():
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    test = pd.read_csv(path + "test.csv")
    train = pd.read_csv(path + "train.csv",encoding = 'ISO-8859-1',low_memory=False )
    address = pd.read_csv(path + "addresses.csv",low_memory=False )
    latlon = pd.read_csv(path + "latlons.csv",low_memory=False )
    
    
    #DROP NAs
    #print ("len before drop NA", len(train) )
    train = train.dropna(subset = ['compliance']).reset_index(drop=True)
    
    def CleanAndTransformData(df):
        #REMOVE FEATURES
        #DROP DATES
        df = df.drop('hearing_date', 1)
        df = df.drop('ticket_issued_date', 1)
        #ADDRESS    
        df = df.drop('mailing_address_str_number', 1)
        df = df.drop('mailing_address_str_name', 1)
        df = df.drop('city', 1)
        df = df.drop('state', 1)
        df = df.drop('zip_code', 1)
        df = df.drop('non_us_str_code', 1)
        df = df.drop('country', 1)
        df = df.drop('violator_name', 1)
        df = df.drop('violation_street_number', 1)
        df = df.drop('violation_street_name', 1)
        #DROP ASSORTED
        df = df.drop('inspector_name', 1)
        df = df.drop('violation_zip_code', 1)
        df = df.drop('clean_up_cost', 1)
        df = df.drop('admin_fee', 1)
        df = df.drop('state_fee', 1)
        df = df.drop('grafitti_status', 1)
        #PERHAPS TOO MANY CATEGORIES
        df = df.drop('violation_code', 1)
        df = df.drop('violation_description', 1)
        #CATEGORIES DON'T MATCH WITH TEST DATA
        df = df.drop('agency_name', 1)
        df = df.drop('disposition', 1)

        #ADD FEATURES
        #INSERT LAT/LON
        lat = latlon.set_index('address')['lat'].to_dict()
        lon = latlon.set_index('address')['lon'].to_dict()
        address["lat"] = address["address"].map(lat)
        address["lon"] = address["address"].map(lon)
        lat = address.set_index('ticket_id')['lat'].to_dict()
        lon = address.set_index('ticket_id')['lon'].to_dict()
        #map lat/lon to dfing dataset
        df["lat"] = df["ticket_id"].map(lat)
        df["lon"] = df["ticket_id"].map(lon)
        
      
        return df
    
    #clean and transform data
    train = CleanAndTransformData(train)
    test = CleanAndTransformData(test)

    #FEATURES NOT IN TEST DATA
    train = train.drop('payment_amount', 1)
    train = train.drop('payment_date', 1)
    train = train.drop('payment_status', 1)
    train = train.drop('balance_due', 1)
    train = train.drop('collection_status', 1)
    train = train.drop('compliance_detail', 1)
    
    #TRAIN AND TEST THE MODEL
    y_train = train["compliance"]
    X_train = train[["fine_amount", "late_fee", "discount_amount", "judgment_amount", "lat", "lon"] ]
    #fillna with mean
    X_train = X_train.fillna(X_train.mean())
    
    X_test = test[["fine_amount", "late_fee", "discount_amount", "judgment_amount", "lat", "lon"] ]

    
    clf = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 6, random_state = 0)
    clf.fit(X_train, y_train)
    
    X_test = X_test.fillna(X_test.mean())
    
    result  =   clf.predict_proba(X_test)
    
    #assign positive class probability predictions
    test["PredProb"] = result[:,1]

    
    test_out = test[["ticket_id", "PredProb"]] 
    
    test_out_series = pd.Series(test_out['PredProb'].values, index=test_out['ticket_id'])
    
    return test_out_series


best_model()

ticket_id
284932    0.049604
285362    0.019340
285361    0.058237
285338    0.062403
285346    0.059115
285345    0.056676
285347    0.073064
285342    0.328722
285530    0.047118
284989    0.029101
285344    0.063772
285343    0.027744
285340    0.029927
285341    0.089976
285349    0.071354
285348    0.068447
284991    0.029101
285532    0.039008
285406    0.028859
285001    0.067540
285006    0.052894
285405    0.018862
285337    0.026078
285496    0.051573
285497    0.056117
285378    0.022079
285589    0.023427
285585    0.051283
285501    0.068152
285581    0.029104
            ...   
376367    0.029976
376366    0.034358
376362    0.035630
376363    0.058806
376365    0.029976
376364    0.034358
376228    0.109724
376265    0.035357
376286    0.314782
376320    0.029752
376314    0.026631
376327    0.379298
376385    0.342050
376435    0.553965
376370    0.345206
376434    0.052685
376459    0.076020
376478    0.007080
376473    0.033018
376484    0.034101
376482    0.034788
37